Dataset Visulization

범주형, 수치형 변수 데이터 분석

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
import numpy as np

# 데이터 로드conda env export > myconda.yaml
file_path = 'G:/인공지능/LG Aimers 6기/온라인 해커톤/data/train.csv'
spec_file_path = 'G:/인공지능/LG Aimers 6기/온라인 해커톤/data/데이터 명세_수정.xlsx'

df = pd.read_csv(file_path)
spec_df = pd.read_excel(spec_file_path)

# 범주형 & 수치형 변수 리스트 추출
categorical_columns = spec_df[spec_df["범주형 여부"] == 1]["컬럼명"].tolist()
numerical_columns = spec_df[spec_df["범주형 여부"] == 0]["컬럼명"].tolist()

print(f"✅ 범주형 변수 개수: {len(categorical_columns)}")
print(f"✅ 수치형 변수 개수: {len(numerical_columns)}")

# 저장할 폴더 생성
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
base_dir = f"G:/인공지능/LG Aimers 6기/온라인 해커톤/data/results_{current_time}"
boxplot_dir = os.path.join(base_dir, "boxplots")  # Boxplot 저장 폴더
category_plot_dir = os.path.join(base_dir, "category_plots")  # 범주형 변수 그래프 저장 폴더
os.makedirs(boxplot_dir, exist_ok=True)
os.makedirs(category_plot_dir, exist_ok=True)

# 🔹 히트맵 (Heatmap) 추가 🔹
plt.figure(figsize=(12, 8))
correlation_matrix = df[numerical_columns].corr()  # 수치형 변수 간 상관관계 계산
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", linewidths=0.5)
plt.title("Feature Correlation Heatmap")
heatmap_path = os.path.join(base_dir, "heatmap.png")
plt.savefig(heatmap_path)
plt.close()

print(f"✅ 히트맵 저장 완료: {heatmap_path}")
# 🔹 결측치 탐색 🔹
missing_values = df.isnull().sum().reset_index()
missing_values.columns = ["컬럼", "결측치 개수"]
missing_values["결측 비율 (%)"] = (missing_values["결측치 개수"] / len(df)) * 100
missing_values = missing_values[missing_values["결측치 개수"] > 0]  # 결측치가 있는 변수만 저장

if not missing_values.empty:
    print("\n✅ 결측치 정보:")
    print(missing_values)

# 🔹 범주형 변수 분석 🔹
categorical_data = {}

for col in categorical_columns:
    cat_counts = df[col].value_counts(dropna=True).reset_index()
    cat_counts.columns = [col, "Count"]
    categorical_data[col] = cat_counts

    # 시각화 (barplot)
    plt.figure(figsize=(8, 4))
    sns.countplot(x=df[col], hue=df['임신 성공 여부'], hue_order=df['임신 성공 여부'].unique(), dodge=True)
    plt.title(f"{col} 분포 및 임신 성공 여부")
    plt.xlabel(col)
    plt.ylabel("Count")
    plt.xticks(rotation=45)

    # 그래프 저장 (category_plots 폴더)
    plot_path = os.path.join(category_plot_dir, f"{col}_category_analysis.png")
    plt.savefig(plot_path)
    plt.close()  # 메모리 절약

# 🔹 수치형 변수 분석 (이상치 포함) 🔹
num_analysis = df[numerical_columns].describe()

# 이상치 탐색 (IQR 방법)
outlier_counts = {}

for col in numerical_columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # 이상치 개수 계산
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    outlier_counts[col] = len(outliers)

    # 이상치 분포 시각화 (Boxplot)
    plt.figure(figsize=(6, 4))
    sns.boxplot(x=df[col])
    plt.title(f"{col} - Boxplot (Outlier Detection)")
    plt.xlabel(col)

    # 그래프 저장 (boxplots 폴더)
    plot_path = os.path.join(boxplot_dir, f"{col}_boxplot.png")
    plt.savefig(plot_path)
    plt.close()  # 메모리 절약

# 이상치 개수 DataFrame 생성
outlier_df = pd.DataFrame(list(outlier_counts.items()), columns=["Feature", "Outlier Count"])

# 🔹 데이터 저장 🔹
output_file = os.path.join(base_dir, "feature_analysis.xlsx")

with pd.ExcelWriter(output_file) as writer:
    if not missing_values.empty:
        missing_values.to_excel(writer, sheet_name="Missing Values", index=False)
    num_analysis.to_excel(writer, sheet_name="Numerical Summary")
    outlier_df.to_excel(writer, sheet_name="Outlier Count", index=False)

    # 범주형 변수별 빈도수 데이터를 개별 시트로 저장
    for col, cat_df in categorical_data.items():
        cat_df.to_excel(writer, sheet_name=f"{col}_Counts", index=False)

print(f"✅ 분석 결과가 저장되었습니다: {base_dir}")
print(f"✅ Boxplot 저장 폴더: {boxplot_dir}")
print(f"✅ 범주형 변수 그래프 저장 폴더: {category_plot_dir}")


✅ 범주형 변수 개수: 48
✅ 수치형 변수 개수: 20


c:\Users\PC\anaconda3\envs\LGAimers6\Lib\site-packages\seaborn\utils.py:61: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from font(s) NanumGothic.
  fig.canvas.draw()
C:\Users\PC\AppData\Local\Temp\ipykernel_30296\2247081811.py:36: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from font(s) NanumGothic.
  plt.savefig(heatmap_path)


✅ 히트맵 저장 완료: G:/인공지능/LG Aimers 6기/온라인 해커톤/data/results_20250208_225923\heatmap.png

✅ 결측치 정보:
                       컬럼  결측치 개수  결측 비율 (%)
3   임신 시도 또는 마지막 임신 경과 연수  246981  96.344855
5                특정 시술 유형       2   0.000780
8             단일 배아 이식 여부    6291   2.454057
9        착상 전 유전 검사 사용 여부  253633  98.939735
10       착상 전 유전 진단 사용 여부    6291   2.454057
28            배아 생성 주요 이유    6291   2.454057
39              총 생성 배아 수    6291   2.454057
40             미세주입된 난자 수    6291   2.454057
41        미세주입에서 생성된 배아 수    6291   2.454057
42               이식된 배아 수    6291   2.454057
43           미세주입 배아 이식 수    6291   2.454057
44               저장된 배아 수    6291   2.454057
45        미세주입 후 저장된 배아 수    6291   2.454057
46               해동된 배아 수    6291   2.454057
47                해동 난자 수    6291   2.454057
48            수집된 신선 난자 수    6291   2.454057
49            저장된 신선 난자 수    6291   2.454057
50               혼합된 난자 수    6291   2.454057
51       파트너 정자와 혼합된 난자 수    6291   2.454057
52    

C:\Users\PC\AppData\Local\Temp\ipykernel_30296\2247081811.py:96: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from font(s) NanumGothic.
  plt.savefig(plot_path)


✅ 분석 결과가 저장되었습니다: G:/인공지능/LG Aimers 6기/온라인 해커톤/data/results_20250208_225923
✅ Boxplot 저장 폴더: G:/인공지능/LG Aimers 6기/온라인 해커톤/data/results_20250208_225923\boxplots
✅ 범주형 변수 그래프 저장 폴더: G:/인공지능/LG Aimers 6기/온라인 해커톤/data/results_20250208_225923\category_plots
